# Finding stars in CLIO images

bditools.findstars_in_dataset takes in a CLIO dataset and finds the (x,y) subpixel location of point sources in the image using DAOStarFinder (https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html).    User supplies the path to a folder containing every skysubtracted image to search (with subscript "skysub" in filename, see cliotools.skysub doc), the integer (x,y) pixel location of a point source (preferably the brightest) in the first image in the list to use as a template, and the number of point sources in the image to search for.  findstars makes a template of the psf of the supplied point source, and for each image: <br>
- performs a cross correlation of the template with the image<br>
- finds the location of the max value in the cross correlation<br>
- uses DAOStarFinder to find the subpixel location of the star at that spot<br>
- masks that star and finds the next largest cross correlation location and uses DAOStarFinder there, then masks that location <br>
- continues to mask and DAOStarFind until the requested number of stars in the image have been found. <br>

When all stars have been found, the script writes the file name and location of all stars in x,y pairs in a file called "StarLocations".<br>
If DAOStarFinder fails to find all stars in the image, either due to poor SNR or distorted psf etc, the missing stars will be marked with nan and the row commented out with #

In [2]:
from cliotools.bditools import findstars_in_dataset
help(findstars_in_dataset)

Help on function findstars_in_dataset in module cliotools.bditools:

findstars_in_dataset(dataset_path, nstars, xca, yca, corrboxsizex=40, corrboxsizey=40, boxsize=100, skip_list=False, append_file=False, threshold=10000.0, radius=20, fwhm=10)
    Find the subpixel location of stars A and B in a clio BDI dataset.
    Parameters:
    -----------
    dataset_path : string
        path to science images including image prefixes and underscores.  
        ex: An image set of target BDI0933 with filenames of the form BDI0933__00xxx.fit
            would take as input a path string of 'BDI0933/BDI0933__'
    xca, yca : int
        integer pixel locations of a star in the first image of the dataset, rough guess 
    corrrboxsize : int
        size of box to draw around star psfs for imstamp and cross correlation
    boxsize : int
        size of box to draw around star psfs for DAOStarFinder
    skip_list : bool
        By default script will make a list of all "skysub" images in given direct

In [1]:
import numpy as np
# Specify path to dataset:
path = 'BD03884A/BD03884A_'
# Supply integer location of bright star in the first image in the dataset,
# which I have saved in a file called "initialguess"
rr = np.loadtxt(path.split('/')[0]+'/initialguess')
xca, yca= rr[0], rr[1]

# Specify parameters:
nstars = 2 # number of stars in image
corrboxsize = 40 # size of box to make image stamp for reference image. Smaller size is better
                 # for run time since it is performing a cross-correlation of a 2d image.
boxsize = 100 # size of box within which DAOStarFinder will search for a star
threshold = 1e4 # expected amplitude of star signal.
radius = 40 # radius for masking stars in image. A few times the fwhm is fine.
fwhm = 20 # approx fwhm for psf in image.

# Findstars in the dataset:
findstars_in_dataset(path, 2, xca, yca, corrboxsizex = corrboxsize, corrboxsizey = corrboxsize,\
                     boxsize = boxsize, skip_list = False, \
                     append_file = False, threshold = threshold, radius = radius, fwhm = fwhm)

188.0 234.0 365.0 154.0
62.0% (8 of 13): |############--------|  Failed to find all stars.
92.0% (12 of 13): |##################--|  Failed to find all stars.
100% (13 of 13): |####################|  Done...
Done


## Reading in output:

In [4]:
# Using pandas (recommended):
readin = pd.read_csv('BD03884A/ABLocations', 
                     delim_whitespace = True, # Spaces separate items in the list
                     comment = '#', # Ignore commented rows
                     names=['filename', 'xca','yca', 'xcb', 'ycb'] # specify column names
                    )
readin

,filename,xca,yca,xcb,ycb
0,BD03884A/BD03884A_00001_skysub.fit,187.742427,234.748694,365.377218,153.967803
1,BD03884A/BD03884A_00002_skysub.fit,187.767799,234.827045,365.441326,154.466378
2,BD03884A/BD03884A_00003_skysub.fit,187.515965,235.327487,365.387360,155.279582
3,BD03884A/BD03884A_00004_skysub.fit,187.589005,235.305617,365.674937,155.601454
4,BD03884A/BD03884A_00005_skysub.fit,187.542786,235.326910,365.742981,155.712430
5,BD03884A/BD03884A_00006_skysub.fit,187.089315,235.513944,365.211311,156.599988
6,BD03884A/BD03884A_00007_skysub.fit,186.943480,235.690810,365.038714,156.659234
7,BD03884A/BD03884A_00008_skysub.fit,186.811133,236.187762,365.514222,157.207327
8,BD03884A/BD03884A_00010_skysub.fit,186.598578,236.147086,365.546560,157.986468
9,BD03884A/BD03884A_00011_skysub.fit,733.767415,288.518923,914.519355,210.608250


In [5]:
# Using Numpy.genfromtxt.  This removes the filenames
readin = np.genfromtxt('BD03884A/ABLocations')
readin

array([[         nan, 187.74242709, 234.74869447, 365.37721801,
        153.96780303],
       [         nan, 187.76779923, 234.82704529, 365.44132552,
        154.46637752],
       [         nan, 187.51596469, 235.32748672, 365.3873596 ,
        155.27958217],
       [         nan, 187.58900482, 235.30561714, 365.67493675,
        155.6014538 ],
       [         nan, 187.54278572, 235.32691019, 365.74298106,
        155.71243004],
       [         nan, 187.0893151 , 235.51394442, 365.21131068,
        156.59998825],
       [         nan, 186.94348047, 235.69080966, 365.03871379,
        156.65923419],
       [         nan, 186.81113341, 236.18776225, 365.51422243,
        157.2073271 ],
       [         nan, 186.59857808, 236.14708568, 365.54656023,
        157.98646762],
       [         nan, 733.76741488, 288.51892257, 914.51935485,
        210.60824996],
       [         nan, 733.19835283, 289.52171037, 914.16077405,
        211.53063605]])

In [10]:
# To preserve column names:
readin = np.genfromtxt('BD03884A/ABLocations', usecols=(1,2,3,4))
readin_names = np.genfromtxt('BD03884A/ABLocations', usecols=(0), dtype=str)
for i in range(len(readin)):
    print(readin_names[i],readin[i])


BD03884A/BD03884A_00001_skysub.fit [187.74242709 234.74869447 365.37721801 153.96780303]
BD03884A/BD03884A_00002_skysub.fit [187.76779923 234.82704529 365.44132552 154.46637752]
BD03884A/BD03884A_00003_skysub.fit [187.51596469 235.32748672 365.3873596  155.27958217]
BD03884A/BD03884A_00004_skysub.fit [187.58900482 235.30561714 365.67493675 155.6014538 ]
BD03884A/BD03884A_00005_skysub.fit [187.54278572 235.32691019 365.74298106 155.71243004]
BD03884A/BD03884A_00006_skysub.fit [187.0893151  235.51394442 365.21131068 156.59998825]
BD03884A/BD03884A_00007_skysub.fit [186.94348047 235.69080966 365.03871379 156.65923419]
BD03884A/BD03884A_00008_skysub.fit [186.81113341 236.18776225 365.51422243 157.2073271 ]
BD03884A/BD03884A_00010_skysub.fit [186.59857808 236.14708568 365.54656023 157.98646762]
BD03884A/BD03884A_00011_skysub.fit [733.76741488 288.51892257 914.51935485 210.60824996]
BD03884A/BD03884A_00012_skysub.fit [733.19835283 289.52171037 914.16077405 211.53063605]
